## 영상 색상처리

In [ ]:
import cv2
import numpy as np
from sklearn.svm import OneClassSVM
import matplotlib as plt

video_path = "C:\\Users\\user\\Pictures\\public\\old_scada.mp4"
cap = cv2.VideoCapture(video_path)


def mask_time(frame):
    frame[54:94,433:445] = 0 # 검은색,  cv2에서는 y 와 x 가 거꾸로 
    frame[298:315,122:310] = 0 # 검은색, cv2에서는 y 와 x 가 거꾸로 
    return frame

def extract_feature(frame):
    frame = mask_time(frame)
    roi = frame[139:196, 129:445]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BAYER_BGR2HSV)

    lower_blue = (80,60,60)
    upper_blue = (110,255,255)

    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    mor = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones(3,3), np.uint8)
  
    feature = mor.flatten() / 255.0
    return feature, mor

train_feature = []
test_feature = []
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret: break

    feature, mor = extract_feature(frame)

    if frame_idx == 313:
        plt.imshow(mor, CMAP = "gray")
        plt.title("fix")
        plt.axis("off")
        plt.show()
        
    if frame_idx <= 828:
        train_feature.append(feature)
    elif 828 < frame_idx < 1337:
        test_feature.append(feature)     
    frame_idx += 1 
cap.release()
X_train = np.array(train_feature)
X_test = np.array(test_feature)

# One - Class SVM 학습
ocsvm = OneClassSVM(kernel="rbf", gamma="auto", nu=0.05)

# 훈련
ocsvm.fit(X_train) 

# 이상 탐지
pred = ocsvm.predict(X_test) # 1:정상 , -1: 비정상
abnormal_idx = np.where(pred==-1)[0] + 828
print("NG(공정 B의 의심)프레임:", abnormal_idx, len(abnormal_idx) )


AttributeError: module 'cv2' has no attribute 'COLOR_BAYER_BGR2HSV'

In [ ]:
import matplotlib.pyplot as plt
cap = cv2.VideoCapture(video_path)


for position in abnormal_idx:
    cap.set(cv2.CAP_PROP_POS_FRAMES, position)
    _, frame = cap.read()
    frame = mask_time(frame)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    plt.subplot(1,2,2)
    plt.imshow(rgb)
    plt.title(f"{position}-frame")
    plt.axis("off")
    plt.show()

